In [1]:
import cobra,rba,lxml
from lxml import etree
import pandas as pd
from collections import OrderedDict

In [2]:
mModel = cobra.io.load_json_model('./input/model/y834_hvd_v3_rba.json')
gs_excl = []

In [3]:
df_prot = pd.read_excel('./curation/prot_stoich_curation.xlsx')
df_prot.index = df_prot.id.to_list()

cofs_all = []
for i in df_prot.index:
    cofs = df_prot.cofactor_stoich[i]
    if pd.isnull(cofs) == False:
        cofs_all += [cof.split(':')[0] for cof in cofs.split(',')]
        
cofs_all = sorted(list(set(cofs_all)))

In [4]:
proteinsET = etree.Element('RBAProteins')

elems = ['listOfComponents', 'listOfMacromolecules']
for e in elems:
    proteinsET.append(etree.Element(e))

In [5]:
aas = 'ACDEFGHIKLMNPQRSTVWY'
aas = [i for i in aas]

comET = proteinsET.find('listOfComponents')
for aa in aas:
    att_dict = OrderedDict({'id':'COM-' + aa, 'type': 'amino_acid', 'weight':'1'})
    comET.append(etree.Element('component', attrib=att_dict))
    
for cof in cofs_all:
    att_dict = OrderedDict({'id':'COM-' + cof, 'type': 'cofactor', 'weight':'0'})
    comET.append(etree.Element('component', attrib=att_dict))

In [7]:
macET = proteinsET.find('listOfMacromolecules')
for i in df_prot.index:
    
    g = df_prot.id[i]
    comp = df_prot.subloc_assigned[i]
    
    att_dict = OrderedDict({'id':'MAC-'+g, 'compartment': 'PART-'+comp})
    macE = etree.SubElement(macET, 'macromolecule', attrib=att_dict)

    prot_seq = df_prot.sequence[i]
    macE_com = etree.SubElement(macE, 'composition')
    for aa in aas:
        att_dict = {'component':'COM-'+aa, 'stoichiometry':str(prot_seq.count(aa))}
        macE_com.append(etree.Element('componentReference', attrib=att_dict))
        
    cofs = df_prot.cofactor_stoich[i]
    if pd.isnull(cofs) == False:
        cof_dict = {c.split(':')[0]:c.split(':')[1] for c in cofs.split(',')}
        keys = sorted(cof_dict.keys())
        for k in keys:
            att_dict = {'component':'COM-'+k, 'stoichiometry':str(cof_dict[k])}
            macE_com.append(etree.Element('componentReference', attrib=att_dict))

In [8]:
with open('./rbaModel/proteins.xml', 'wb') as f:
    f.write(etree.tostring(proteinsET, pretty_print=True))